In [341]:
import pandas as pd
import requests
import json
import datetime

In [342]:
gare = "Paris - Montparnasse - Hall 1 & 2"

# Max 250
nb_trains = 20

In [343]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [344]:
dic_gare = {"Paris - Montparnasse - Hall 1 & 2": "SNCF:87391003",
            "Paris - Gare de Lyon - Hall 1 & 2": "SNCF:87686006",
            "Paris Est": "SNCF:87113001",
            "Paris Nord": "SNCF:87271007",
            "Lyon Part Dieu": "SNCF:87723197",
            "Rennes": "SNCF:87471003",
            "Bordeaux Saint-Jean": "SNCF:87581009",
            "Nantes": "SNCF:87481002",
            "Montpellier Saint-Roch": "SNCF:87773002",
            "Nîmes Centre": "SNCF:87775007",
            "Strasbourg": "SNCF:87212027",
            "Marseille - Saint-Charles": "SNCF:87751008",
            "Lille Flandres": "SNCF:87286005",
            "Toulouse Matabiau": "SNCF:87611004",
            "Alès": "SNCF:87775288"
            }

In [345]:
link = 'https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:' + dic_gare[gare] + '/departures?count=' + str(nb_trains)
req = requests.get(link,auth=(token, ''))
#print(link)

In [346]:
doc = json.loads(req.text)
row = len(doc['departures'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 20


In [347]:
df = pd.DataFrame(doc['departures'])
df_gare = pd.DataFrame(list(df['display_informations']))
df_heure = pd.DataFrame(list(df['stop_date_time']))
df_id = pd.DataFrame(list(df['links']))
df_id = pd.DataFrame(list(df_id[1]))

In [348]:
def del_day(string):
    return string[9:]

df_heure['departure_date_time'] = df_heure['departure_date_time'].apply(del_day)
df_heure['base_departure_date_time'] = df_heure['base_departure_date_time'].apply(del_day)

In [349]:
def conv_min(string):
    return int(string[0:2])*60 + int(string[2:4])

df_heure['retard'] = df_heure['departure_date_time'].apply(conv_min) - df_heure['base_departure_date_time'].apply(conv_min)

In [350]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_heure['heure'] = df_heure['departure_date_time'].apply(str_tps)
df_heure['old_heure'] = df_heure['base_departure_date_time'].apply(str_tps)

In [351]:
def del_par(string):
    index = string.find("(")
    return string[:index]

df_gare['direction'] = df_gare['direction'].apply(del_par)

In [352]:
df_gare = df_gare[['direction','network','trip_short_name']]
df_gare.rename(columns = {'direction':'Destination'}, inplace = True)
df_gare.rename(columns = {'network':'Train'}, inplace = True)
df_gare.rename(columns = {'trip_short_name':'Numéro'}, inplace = True)

df_gare['Départ (réel)'] = df_heure['heure']
df_gare['Départ (prévu)'] = df_heure['old_heure']
df_gare['Retard (min)'] = df_heure['retard']
df_gare['id'] = df_id['id']

In [353]:
def get_name(string):
    string = string[10:]
    index_fin = string.find("', 'links'")
    return string[:index_fin]

arrets = []
causes = []

for i in df_gare.index:
    id = df_gare.iloc[i]['id']
    
    if 'RealTime' in id:
        index_id = id.index("RealTime")
        id = id[:index_id-1]

    link_voyage = 'https://api.sncf.com/v1/coverage/sncf/vehicle_journeys/' + id
    req_arret = requests.get(link_voyage ,auth=(token, ''))
    doc_voyage = json.loads(req_arret.text)

    df_arret = pd.DataFrame(doc_voyage['vehicle_journeys'])
    df_arret = pd.DataFrame(list(df_arret['stop_times']))
    df_arret = df_arret.T
    df_arret = pd.DataFrame(list(df_arret[0]))
    df_arret['stop_point'] = df_arret['stop_point'].astype('str').apply(get_name)

    if df_gare.iloc[i]['Retard (min)'] != 0:
        df_retard = pd.DataFrame(doc_voyage['disruptions'])
        df_retard = pd.DataFrame(list(df_retard['messages'][0]))
        causes.append(df_retard.iloc[0]['text'])
    else:
        causes.append("")

    liste_arrets = list(df_arret['stop_point'])
    if gare not in liste_arrets:
        if "Paris Gare de Lyon" in liste_arrets:
            index_gare = liste_arrets.index("Paris Gare de Lyon")
        if "Paris - Gare de Lyon - Banlieue" in liste_arrets:
            index_gare = liste_arrets.index("Paris - Gare de Lyon - Banlieue")
        if "Gare du Nord Surface" in liste_arrets:
            index_gare = liste_arrets.index("Gare du Nord Surface")
    else:
        index_gare = liste_arrets.index(gare)

    liste_arrets = liste_arrets[index_gare+1:]

    arrets.append(liste_arrets)

In [354]:
df_gare['Cause'] = causes
df_gare['Arrêts'] = arrets
df_gare = df_gare.drop(['id'], axis=1)

In [355]:
df_gare

,Destination,Train,Numéro,Départ (réel),Départ (prévu),Retard (min),Cause,Arrêts
0,Rambouillet,TRANSILIEN,165405,12:35:00,12:35:00,0,,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
1,Le Croisic,TGV INOUI,8921,12:36:00,12:36:00,0,,"[Angers-Saint-Laud, Nantes, Saint-Nazaire, La ..."
2,Mantes-la-Jolie,TRANSILIEN,165607,12:50:00,12:50:00,0,,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
3,Brest,TGV INOUI,8619,12:56:00,12:56:00,0,,"[Rennes, Saint-Brieuc, Brest]"
4,Dreux,TRANSILIEN,165807,12:58:00,12:58:00,0,,"[Versailles Chantiers, Plaisir - Grignon, Vill..."
5,Quimper,OUIGO,7615,13:04:30,12:44:30,20,Mise à quai tardive en gare origine,"[Le Mans, Laval, Rennes, Vannes, Auray, Lorien..."
6,Rambouillet,TRANSILIEN,165409,13:05:00,13:05:00,0,,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
7,Le Mans,TER,16765,13:06:00,13:06:00,0,,"[Versailles Chantiers, Rambouillet, Épernon, M..."
8,Plaisir - Grignon,TRANSILIEN,165511,13:20:00,13:20:00,0,,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
9,Nantes,TGV INOUI,8875,13:29:00,13:29:00,0,,"[Le Mans, Sablé-sur-Sarthe, Angers-Saint-Laud,..."
